In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from urllib import request
from urllib.error import HTTPError
import re
from google.colab  import drive
drive.mount("/gdrive")
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [2]:
directory_path = "/gdrive/My Drive/Yedek/ThermalDatabase"
save_path='/gdrive/My Drive/Yedek/Thermal_Image_test'

In [3]:
def convertImage(target,destination,delimiter=None):
   m1=np.loadtxt(target,dtype='float',delimiter=delimiter)
   plt.set_cmap('viridis')
   plt.imsave(destination,m1)


In [4]:
def log_to_file(text_input):
  with open('/gdrive/My Drive/Yedek/log.txt', 'a') as f:
        for text in text_input:
          f.write(text)
          f.write('\n')

# **Create a recursive function to walk subdirectories**

In [5]:
def walk_subdirectories(d_path,s_path):
   subdirectories,files=next(os.walk(d_path))[1:]
   if(len(files)==0):
      for subdirectory in subdirectories:
        os.mkdir(s_path+'/'+subdirectory)
        walk_subdirectories(d_path+'/'+subdirectory,s_path+'/'+subdirectory)
   else:
    for file in files:
      patient_id=re.split('[A-Z]',file.split(".")[0])[1]
      t=d_path+'/'+file
      filename=file.split('txt')[0]
      d=s_path+'/'+filename+'.'+'jpg'
      try:
        convertImage(t,d)
      except ValueError as v1:
        log_to_file(["File is not OK... Try delimiter ';'","Filename",str(file),"Patient ID",patient_id])
        try:
          convertImage(t,d,delimiter=";")
          log_to_file(["File is now OK... delimiter is ';' !!!!!!!","Filename",str(file),"Patient ID",patient_id])
        except ValueError as v2:
          log_to_file(["File still not OK, Try to download again...","Filename",str(file),"Patient ID",patient_id])
          d_link=f"http://visual.ic.uff.br/dmi/bancovl/{patient_id}/"
          try:
           request.urlretrieve(d_link+file,t)
           try:
            convertImage(t,d)
            log_to_file(["File is now OK after download...","Filename",str(file),"Patient ID",patient_id])
           except ValueError as v3:
            log_to_file(["File is not OK after download... Try delimiter ';' ","Filename",str(file),"Patient ID",patient_id])
            try:
             convertImage(t,d,delimiter=";")
             log_to_file(["File is now OK after download... delimiter is ';' !!!!!!!","Filename",str(file),"Patient ID",patient_id])
            except ValueError as v4:
             log_to_file(["'Very exceptional case'!!!! File is not ok even after download and delimiter....:SS","Filename",str(file),"Patient ID",patient_id])
          except  HTTPError as h:
           log_to_file(["No file exists, HTTP 404 Error!!!!!!!","Patient ID",patient_id])




# **Initiate Recursion**

In [6]:
walk_subdirectories(directory_path,save_path)

<Figure size 640x480 with 0 Axes>